In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tifffile as tiff
import keras
from keras import layers
from tensorflow import data as tf_data
import tensorflow as tf

In [2]:
############ORGANIZE DATABASE##############################
import shutil
import os

def move_file(src_fold, dest_fold, filename):
    source_path = os.path.join(src_fold, filename)
    destination_path = os.path.join(dest_fold, filename)
    try:
        shutil.move(source_path, destination_path)
    except FileNotFoundError:
        print(f"File '{filename}' not found in '{src_fold}'.")
    except PermissionError:
        print(f"Permission denied to move file '{filename}'.")

src_fold = './right_png'
dest_fold = './plaice_right_png/'

In [ ]:
################################################ MODEL #############################################################

In [5]:
#Importing and Scaling data
n_data=keras.utils.image_dataset_from_directory(directory='./plaice_right_png',
color_mode="rgb",
image_size=(256, 256),#scaling data
)
#spliting data
train_size=int(len(n_data)*.7)
val_size=int(len(n_data)*.2)+1
test_size=int(len(n_data)*.1)+1
n_data=n_data.map(lambda x,y:(x/255,y))
n_train=n_data.take(train_size)
n_val=n_data.skip(train_size).take(val_size)
n_test=n_data.skip(train_size+val_size).take(test_size)

#training data
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  
        previous_block_activation = x  

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    outputs = layers.Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=(256,256,3), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

epochs = 1

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)
model.fit(
    n_train,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=n_val,
)

Found 1057 files belonging to 2 classes.
You must install pydot (`pip install pydot`) for `plot_model` to work.
23/23 ━━━━━━━━━━━━━━━━━━━━ 299s 13s/step - acc: 0.6789 - loss: 0.5711 - val_acc: 0.5179 - val_loss: 0.6931
